## 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from bs4 import BeautifulSoup
import folium

In [ ]:
# 한글깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
# Matplotlib에 나눔바른고딕 폰트 설정
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지

## 기준 데이터

In [ ]:
# 병합할 기준 데이터 불러오기
data = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/행정구역/천안시 행정구역.csv')

In [ ]:
data = data[['구','구분','이름']]

In [ ]:
data.head()

,구,구분,이름
0,동남구,읍·면,목천읍
1,동남구,읍·면,풍세면
2,동남구,읍·면,광덕면
3,동남구,읍·면,북면
4,동남구,읍·면,성남면


## 1.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_고령화지수_소윤.csv')

In [ ]:
a.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트)
0,동남구,읍·면,목천읍,22292,4924,22.1
1,동남구,읍·면,풍세면,10401,2044,19.7
2,동남구,읍·면,광덕면,4004,1722,43.0
3,동남구,읍·면,북면,4109,1641,39.9
4,동남구,읍·면,성남면,2812,1281,45.6


In [ ]:
a_subset = a.iloc[:, 2:]

In [ ]:
a_subset.head()

,이름,전체,65세 이상,고령화율(단위: 퍼센트)
0,목천읍,22292,4924,22.1
1,풍세면,10401,2044,19.7
2,광덕면,4004,1722,43.0
3,북면,4109,1641,39.9
4,성남면,2812,1281,45.6


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트)
0,동남구,읍·면,목천읍,22292,4924,22.1
1,동남구,읍·면,풍세면,10401,2044,19.7
2,동남구,읍·면,광덕면,4004,1722,43.0
3,동남구,읍·면,북면,4109,1641,39.9
4,동남구,읍·면,성남면,2812,1281,45.6


## 2.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_시설수_은주.csv')

In [ ]:
a.head()

,Unnamed: 0,구,구분,이름,노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,노인1천명당_여가복지시설수_노인교실,노인1천명당_노인돌봄서비스수행기관수,노인1천명당_무료급식소수,노인1천명당_치매안심센터수,노인1천명당_자립형일자리수행기관수
0,0,동남구,읍·면,목천읍,2.233956,0.406174,1.421608,8.529651,0.00000,0.0,0.0,0.0,0.00000
1,1,동남구,읍·면,풍세면,2.935421,0.000000,2.446184,12.230920,0.00000,0.0,0.0,0.0,0.00000
2,2,동남구,읍·면,광덕면,1.742160,0.000000,1.161440,21.486643,0.58072,0.0,0.0,0.0,0.58072
3,3,동남구,읍·면,북면,1.828154,0.000000,0.609385,15.234613,0.00000,0.0,0.0,0.0,0.00000
4,4,동남구,읍·면,성남면,2.341920,0.000000,0.000000,23.419204,0.00000,0.0,0.0,0.0,0.00000


In [ ]:
a_subset = a.iloc[:, 3:]

In [ ]:
a_subset.head()

,이름,노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,노인1천명당_여가복지시설수_노인교실,노인1천명당_노인돌봄서비스수행기관수,노인1천명당_무료급식소수,노인1천명당_치매안심센터수,노인1천명당_자립형일자리수행기관수
0,목천읍,2.233956,0.406174,1.421608,8.529651,0.00000,0.0,0.0,0.0,0.00000
1,풍세면,2.935421,0.000000,2.446184,12.230920,0.00000,0.0,0.0,0.0,0.00000
2,광덕면,1.742160,0.000000,1.161440,21.486643,0.58072,0.0,0.0,0.0,0.58072
3,북면,1.828154,0.000000,0.609385,15.234613,0.00000,0.0,0.0,0.0,0.00000
4,성남면,2.341920,0.000000,0.000000,23.419204,0.00000,0.0,0.0,0.0,0.00000


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,노인1천명당_여가복지시설수_노인교실,노인1천명당_노인돌봄서비스수행기관수,노인1천명당_무료급식소수,노인1천명당_치매안심센터수,노인1천명당_자립형일자리수행기관수
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,0.00000,0.0,0.0,0.0,0.00000
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,0.00000,0.0,0.0,0.0,0.00000
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,0.58072,0.0,0.0,0.0,0.58072
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,0.00000,0.0,0.0,0.0,0.00000
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,0.00000,0.0,0.0,0.0,0.00000


## 3.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_재희.csv')

In [ ]:
a.head()

,구,구분,이름,고령자비율,고령자밀집도지수,복지시설다양성지수,교통취약노인수,복지서비스잠재수요지수
0,동남구,읍·면,목천읍,0.220886,0.437636,0.333333,1681,0.109155
1,동남구,읍·면,풍세면,0.196520,0.389358,0.133333,3111,0.113279
2,동남구,읍·면,광덕면,0.430070,0.852084,0.466667,0,0.054533
3,동남구,읍·면,북면,0.399367,0.791254,0.066667,0,0.363777
4,동남구,읍·면,성남면,0.455548,0.902563,0.266667,424,0.070993


In [ ]:
a_subset = a.iloc[:, 2:]

In [ ]:
a_subset.head()

,이름,고령자비율,고령자밀집도지수,복지시설다양성지수,교통취약노인수,복지서비스잠재수요지수
0,목천읍,0.220886,0.437636,0.333333,1681,0.109155
1,풍세면,0.196520,0.389358,0.133333,3111,0.113279
2,광덕면,0.430070,0.852084,0.466667,0,0.054533
3,북면,0.399367,0.791254,0.066667,0,0.363777
4,성남면,0.455548,0.902563,0.266667,424,0.070993


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,노인1천명당_여가복지시설수_노인교실,노인1천명당_노인돌봄서비스수행기관수,노인1천명당_무료급식소수,노인1천명당_치매안심센터수,노인1천명당_자립형일자리수행기관수,고령자비율,고령자밀집도지수,복지시설다양성지수,교통취약노인수,복지서비스잠재수요지수
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,0.00000,0.0,0.0,0.0,0.00000,0.220886,0.437636,0.333333,1681,0.109155
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,0.00000,0.0,0.0,0.0,0.00000,0.196520,0.389358,0.133333,3111,0.113279
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,0.58072,0.0,0.0,0.0,0.58072,0.430070,0.852084,0.466667,0,0.054533
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,0.00000,0.0,0.0,0.0,0.00000,0.399367,0.791254,0.066667,0,0.363777
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,0.00000,0.0,0.0,0.0,0.00000,0.455548,0.902563,0.266667,424,0.070993


## 4.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_지역별_보행노인사고_수.csv')

In [ ]:
a.head()

,구,구분,이름,사고건수
0,동남구,읍·면,목천읍,0
1,동남구,읍·면,풍세면,0
2,동남구,읍·면,광덕면,0
3,동남구,읍·면,북면,0
4,동남구,읍·면,성남면,0


In [ ]:
a_subset = a.iloc[:, 2:]

In [ ]:
a_subset.head()

,이름,사고건수
0,목천읍,0
1,풍세면,0
2,광덕면,0
3,북면,0
4,성남면,0


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,노인1천명당_노인돌봄서비스수행기관수,노인1천명당_무료급식소수,노인1천명당_치매안심센터수,노인1천명당_자립형일자리수행기관수,고령자비율,고령자밀집도지수,복지시설다양성지수,교통취약노인수,복지서비스잠재수요지수,사고건수
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.0,0.0,0.0,0.00000,0.220886,0.437636,0.333333,1681,0.109155,0
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.0,0.0,0.0,0.00000,0.196520,0.389358,0.133333,3111,0.113279,0
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.0,0.0,0.0,0.58072,0.430070,0.852084,0.466667,0,0.054533,0
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.0,0.0,0.0,0.00000,0.399367,0.791254,0.066667,0,0.363777,0
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.0,0.0,0.0,0.00000,0.455548,0.902563,0.266667,424,0.070993,0


## 5.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_지역별_보호구역_수_소윤.csv')

In [ ]:
a.head()

,구,구분,이름,보호구역 수
0,동남구,읍·면,목천읍,3
1,동남구,읍·면,풍세면,0
2,동남구,읍·면,광덕면,7
3,동남구,읍·면,북면,1
4,동남구,읍·면,성남면,2


In [ ]:
a_subset = a.iloc[:, 2:]

In [ ]:
a_subset.head()

,이름,보호구역 수
0,목천읍,3
1,풍세면,0
2,광덕면,7
3,북면,1
4,성남면,2


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,노인1천명당_무료급식소수,노인1천명당_치매안심센터수,노인1천명당_자립형일자리수행기관수,고령자비율,고령자밀집도지수,복지시설다양성지수,교통취약노인수,복지서비스잠재수요지수,사고건수,보호구역 수
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.0,0.0,0.00000,0.220886,0.437636,0.333333,1681,0.109155,0,3
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.0,0.0,0.00000,0.196520,0.389358,0.133333,3111,0.113279,0,0
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.0,0.0,0.58072,0.430070,0.852084,0.466667,0,0.054533,0,7
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.0,0.0,0.00000,0.399367,0.791254,0.066667,0,0.363777,0,1
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.0,0.0,0.00000,0.455548,0.902563,0.266667,424,0.070993,0,2


## 6.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_고령 경제활동인구.csv')

In [ ]:
a.head()

,구,구분,이름,기준연월,월경제활동인구수_65_69,월경제활동인구수_70세이상
0,동남구,읍·면,광덕면,202310,372.0,509.0
1,동남구,읍·면,동면,202310,173.0,301.0
2,동남구,읍·면,목천읍,202310,1197.0,1531.0
3,동남구,읍·면,병천면,202310,460.0,564.0
4,동남구,읍·면,북면,202310,386.0,504.0


In [ ]:
a_subset = a.iloc[:, 2:]

In [ ]:
a_subset.head()

,이름,기준연월,월경제활동인구수_65_69,월경제활동인구수_70세이상
0,광덕면,202310,372.0,509.0
1,동면,202310,173.0,301.0
2,목천읍,202310,1197.0,1531.0
3,병천면,202310,460.0,564.0
4,북면,202310,386.0,504.0


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,고령자비율,고령자밀집도지수,복지시설다양성지수,교통취약노인수,복지서비스잠재수요지수,사고건수,보호구역 수,기준연월,월경제활동인구수_65_69,월경제활동인구수_70세이상
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.220886,0.437636,0.333333,1681,0.109155,0,3,202310,1197.0,1531.0
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.196520,0.389358,0.133333,3111,0.113279,0,0,202310,354.0,418.0
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.430070,0.852084,0.466667,0,0.054533,0,7,202310,372.0,509.0
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.399367,0.791254,0.066667,0,0.363777,0,1,202310,386.0,504.0
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.455548,0.902563,0.266667,424,0.070993,0,2,202310,291.0,359.0


## 7.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_고령 대출평균.csv')

In [ ]:
a.head()

,구,구분,이름,기준연월,월대출평균잔액_65_69,월대출평균잔액_70세이상
0,동남구,읍·면,광덕면,202310,147863.0,147221.0
1,동남구,읍·면,동면,202310,164984.0,96465.0
2,동남구,읍·면,목천읍,202310,130521.0,117930.0
3,동남구,읍·면,병천면,202310,137119.0,203354.0
4,동남구,읍·면,북면,202310,143114.0,161804.0


In [ ]:
a_subset = a.iloc[:, [2,4,5]]

In [ ]:
a_subset.head()

,이름,월대출평균잔액_65_69,월대출평균잔액_70세이상
0,광덕면,147863.0,147221.0
1,동면,164984.0,96465.0
2,목천읍,130521.0,117930.0
3,병천면,137119.0,203354.0
4,북면,143114.0,161804.0


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,복지시설다양성지수,교통취약노인수,복지서비스잠재수요지수,사고건수,보호구역 수,기준연월,월경제활동인구수_65_69,월경제활동인구수_70세이상,월대출평균잔액_65_69,월대출평균잔액_70세이상
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.333333,1681,0.109155,0,3,202310,1197.0,1531.0,130521.0,117930.0
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.133333,3111,0.113279,0,0,202310,354.0,418.0,222727.0,241594.0
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.466667,0,0.054533,0,7,202310,372.0,509.0,147863.0,147221.0
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.066667,0,0.363777,0,1,202310,386.0,504.0,143114.0,161804.0
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.266667,424,0.070993,0,2,202310,291.0,359.0,218323.0,138515.0


## 8.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_고령 소득평균.csv')

In [ ]:
a.head()

,구,구분,이름,기준연월,월소득평균_65_69,월소득평균_70세이상
0,동남구,읍·면,광덕면,202310,4790.0,4285.0
1,동남구,읍·면,동면,202310,4883.0,4348.0
2,동남구,읍·면,목천읍,202310,4742.0,4150.0
3,동남구,읍·면,병천면,202310,4813.0,4253.0
4,동남구,읍·면,북면,202310,5328.0,4194.0


In [ ]:
a_subset = a.iloc[:, [2,4,5]]

In [ ]:
a_subset.head()

,이름,월소득평균_65_69,월소득평균_70세이상
0,광덕면,4790.0,4285.0
1,동면,4883.0,4348.0
2,목천읍,4742.0,4150.0
3,병천면,4813.0,4253.0
4,북면,5328.0,4194.0


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,복지서비스잠재수요지수,사고건수,보호구역 수,기준연월,월경제활동인구수_65_69,월경제활동인구수_70세이상,월대출평균잔액_65_69,월대출평균잔액_70세이상,월소득평균_65_69,월소득평균_70세이상
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.109155,0,3,202310,1197.0,1531.0,130521.0,117930.0,4742.0,4150.0
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.113279,0,0,202310,354.0,418.0,222727.0,241594.0,5108.0,4734.0
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.054533,0,7,202310,372.0,509.0,147863.0,147221.0,4790.0,4285.0
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.363777,0,1,202310,386.0,504.0,143114.0,161804.0,5328.0,4194.0
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.070993,0,2,202310,291.0,359.0,218323.0,138515.0,5373.0,4201.0


## 9.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_고령 신용평점평균.csv')

In [ ]:
a.head()

,구,구분,이름,기준연월,월신용평점평균_65_69,월신용평점평균_70세이상
0,동남구,읍·면,광덕면,202310,1629.0,1562.0
1,동남구,읍·면,동면,202310,1591.0,1560.0
2,동남구,읍·면,목천읍,202310,1628.0,1570.0
3,동남구,읍·면,병천면,202310,1638.0,1555.0
4,동남구,읍·면,북면,202310,1674.0,1556.0


In [ ]:
a_subset = a.iloc[:, [2,4,5]]

In [ ]:
a_subset.head()

,이름,월신용평점평균_65_69,월신용평점평균_70세이상
0,광덕면,1629.0,1562.0
1,동면,1591.0,1560.0
2,목천읍,1628.0,1570.0
3,병천면,1638.0,1555.0
4,북면,1674.0,1556.0


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,보호구역 수,기준연월,월경제활동인구수_65_69,월경제활동인구수_70세이상,월대출평균잔액_65_69,월대출평균잔액_70세이상,월소득평균_65_69,월소득평균_70세이상,월신용평점평균_65_69,월신용평점평균_70세이상
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,3,202310,1197.0,1531.0,130521.0,117930.0,4742.0,4150.0,1628.0,1570.0
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0,202310,354.0,418.0,222727.0,241594.0,5108.0,4734.0,1631.0,1557.0
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,7,202310,372.0,509.0,147863.0,147221.0,4790.0,4285.0,1629.0,1562.0
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,1,202310,386.0,504.0,143114.0,161804.0,5328.0,4194.0,1674.0,1556.0
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,2,202310,291.0,359.0,218323.0,138515.0,5373.0,4201.0,1625.0,1549.0


## 10.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_고령 카드소비평균.csv')

In [ ]:
a.head()

,구,구분,이름,기준연월,월카드총이용금액평균_65_69,월카드총이용금액평균_70세이상
0,동남구,읍·면,광덕면,202310,2080.0,926.0
1,동남구,읍·면,동면,202310,1833.0,982.0
2,동남구,읍·면,목천읍,202310,2263.0,1121.0
3,동남구,읍·면,병천면,202310,2003.0,975.0
4,동남구,읍·면,북면,202310,2425.0,897.0


In [ ]:
a_subset = a.iloc[:, [2,4,5]]

In [ ]:
a_subset.head()

,이름,월카드총이용금액평균_65_69,월카드총이용금액평균_70세이상
0,광덕면,2080.0,926.0
1,동면,1833.0,982.0
2,목천읍,2263.0,1121.0
3,병천면,2003.0,975.0
4,북면,2425.0,897.0


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,월경제활동인구수_65_69,월경제활동인구수_70세이상,월대출평균잔액_65_69,월대출평균잔액_70세이상,월소득평균_65_69,월소득평균_70세이상,월신용평점평균_65_69,월신용평점평균_70세이상,월카드총이용금액평균_65_69,월카드총이용금액평균_70세이상
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,1197.0,1531.0,130521.0,117930.0,4742.0,4150.0,1628.0,1570.0,2263.0,1121.0
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,354.0,418.0,222727.0,241594.0,5108.0,4734.0,1631.0,1557.0,2463.0,774.0
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,372.0,509.0,147863.0,147221.0,4790.0,4285.0,1629.0,1562.0,2080.0,926.0
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,386.0,504.0,143114.0,161804.0,5328.0,4194.0,1674.0,1556.0,2425.0,897.0
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,291.0,359.0,218323.0,138515.0,5373.0,4201.0,1625.0,1549.0,2095.0,837.0


## 11.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_노인 경제자립도 지수.csv')

In [ ]:
a.head()

,구,구분,이름,경제활동참가율_65_69,경제활동참가율_70세이상,고령인구수_65_69,고령인구수_70세이상,월경제활동인구수_65_69,월경제활동인구수_70세이상,경제활동참가율_전체
0,동남구,읍·면,광덕면,0.667864,0.436910,557.0,1165.0,372.0,509.0,0.511614
1,동남구,읍·면,동면,0.681102,0.425743,254.0,707.0,173.0,301.0,0.493236
2,동남구,읍·면,목천읍,0.697146,0.477393,1717.0,3207.0,1197.0,1531.0,0.554021
3,동남구,읍·면,병천면,0.703364,0.435858,654.0,1294.0,460.0,564.0,0.525667
4,동남구,읍·면,북면,0.654237,0.479543,590.0,1051.0,386.0,504.0,0.542352


In [ ]:
a_subset = a.iloc[:, 2:]

In [ ]:
a_subset.head()

,이름,경제활동참가율_65_69,경제활동참가율_70세이상,고령인구수_65_69,고령인구수_70세이상,월경제활동인구수_65_69,월경제활동인구수_70세이상,경제활동참가율_전체
0,광덕면,0.667864,0.436910,557.0,1165.0,372.0,509.0,0.511614
1,동면,0.681102,0.425743,254.0,707.0,173.0,301.0,0.493236
2,목천읍,0.697146,0.477393,1717.0,3207.0,1197.0,1531.0,0.554021
3,병천면,0.703364,0.435858,654.0,1294.0,460.0,564.0,0.525667
4,북면,0.654237,0.479543,590.0,1051.0,386.0,504.0,0.542352


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,월신용평점평균_70세이상,월카드총이용금액평균_65_69,월카드총이용금액평균_70세이상,경제활동참가율_65_69,경제활동참가율_70세이상,고령인구수_65_69,고령인구수_70세이상,월경제활동인구수_65_69_y,월경제활동인구수_70세이상_y,경제활동참가율_전체
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,1570.0,2263.0,1121.0,0.697146,0.477393,1717.0,3207.0,1197.0,1531.0,0.554021
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,1557.0,2463.0,774.0,0.452107,0.331483,783.0,1261.0,354.0,418.0,0.377691
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,1562.0,2080.0,926.0,0.667864,0.436910,557.0,1165.0,372.0,509.0,0.511614
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,1556.0,2425.0,897.0,0.654237,0.479543,590.0,1051.0,386.0,504.0,0.542352
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,1549.0,2095.0,837.0,0.751938,0.401566,387.0,894.0,291.0,359.0,0.507416


## 12.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_노인 금융취약도 지수.csv')

In [ ]:
a.head()

,구,구분,이름,월소득평균,월대출평균잔액,월신용평점평균,금융취약도지수
0,동남구,읍·면,광덕면,2078.319149,73207.087766,779.869681,35.444
1,동남구,읍·면,동면,2055.595745,52996.252660,771.550532,26.010
2,동남구,읍·면,목천읍,2033.146277,64613.103723,783.196809,31.997
3,동남구,읍·면,병천면,2151.909574,86436.335106,774.287234,40.393
4,동남구,읍·면,북면,2078.478723,76142.292553,779.638298,36.854


In [ ]:
a_subset = a.iloc[:, 2:]

In [ ]:
a_subset.head()

,이름,월소득평균,월대출평균잔액,월신용평점평균,금융취약도지수
0,광덕면,2078.319149,73207.087766,779.869681,35.444
1,동면,2055.595745,52996.252660,771.550532,26.010
2,목천읍,2033.146277,64613.103723,783.196809,31.997
3,병천면,2151.909574,86436.335106,774.287234,40.393
4,북면,2078.478723,76142.292553,779.638298,36.854


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,경제활동참가율_70세이상,고령인구수_65_69,고령인구수_70세이상,월경제활동인구수_65_69_y,월경제활동인구수_70세이상_y,경제활동참가율_전체,월소득평균,월대출평균잔액,월신용평점평균,금융취약도지수
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.477393,1717.0,3207.0,1197.0,1531.0,0.554021,2033.146277,64613.103723,783.196809,31.997
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.331483,783.0,1261.0,354.0,418.0,0.377691,2294.994681,114478.763298,783.319149,50.099
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.436910,557.0,1165.0,372.0,509.0,0.511614,2078.319149,73207.087766,779.869681,35.444
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.479543,590.0,1051.0,386.0,504.0,0.542352,2078.478723,76142.292553,779.638298,36.854
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.401566,387.0,894.0,291.0,359.0,0.507416,2095.970745,74372.853723,774.949468,35.709


## 13.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_독거노인_2019_경제수준_비율.csv')

In [ ]:
a.head()

,구,이름,국민기초생활보장_독거노인_수급자비율,저소득_독거노인_비율,일반_독거노인_비율
0,동남구,광덕면,0.113839,0.049107,0.837054
1,동남구,동면,0.094737,0.031579,0.873684
2,동남구,목천읍,0.177093,0.059912,0.762996
3,동남구,문성동,0.215753,0.044521,0.739726
4,동남구,병천면,0.168016,0.070850,0.761134


In [ ]:
a_subset = a.iloc[:, 1:]

In [ ]:
a_subset.head()

,이름,국민기초생활보장_독거노인_수급자비율,저소득_독거노인_비율,일반_독거노인_비율
0,광덕면,0.113839,0.049107,0.837054
1,동면,0.094737,0.031579,0.873684
2,목천읍,0.177093,0.059912,0.762996
3,문성동,0.215753,0.044521,0.739726
4,병천면,0.168016,0.070850,0.761134


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,월경제활동인구수_65_69_y,월경제활동인구수_70세이상_y,경제활동참가율_전체,월소득평균,월대출평균잔액,월신용평점평균,금융취약도지수,국민기초생활보장_독거노인_수급자비율,저소득_독거노인_비율,일반_독거노인_비율
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,1197.0,1531.0,0.554021,2033.146277,64613.103723,783.196809,31.997,0.177093,0.059912,0.762996
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,354.0,418.0,0.377691,2294.994681,114478.763298,783.319149,50.099,0.100575,0.106322,0.793103
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,372.0,509.0,0.511614,2078.319149,73207.087766,779.869681,35.444,0.113839,0.049107,0.837054
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,386.0,504.0,0.542352,2078.478723,76142.292553,779.638298,36.854,0.125604,0.130435,0.743961
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,291.0,359.0,0.507416,2095.970745,74372.853723,774.949468,35.709,0.090909,0.038961,0.870130


## 14.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_독거노인_2019_수급자_비율.csv')

In [ ]:
a.head()

,구,이름,국민기초생활보장_독거노인_수급자비율,국민기초생활보장_65~79세_수급자비율,국민기초생활보장_80세이상_수급자비율
0,동남구,목천읍,0.177093,0.173333,0.184416
1,동남구,풍세면,0.100575,0.105991,0.091603
2,동남구,광덕면,0.113839,0.111913,0.116959
3,동남구,북면,0.125604,0.124060,0.128378
4,동남구,성남면,0.090909,0.072917,0.120690


In [ ]:
a_subset = a.iloc[:, 1:]

In [ ]:
a_subset.head()

,이름,국민기초생활보장_독거노인_수급자비율,국민기초생활보장_65~79세_수급자비율,국민기초생활보장_80세이상_수급자비율
0,목천읍,0.177093,0.173333,0.184416
1,풍세면,0.100575,0.105991,0.091603
2,광덕면,0.113839,0.111913,0.116959
3,북면,0.125604,0.124060,0.128378
4,성남면,0.090909,0.072917,0.120690


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,월소득평균,월대출평균잔액,월신용평점평균,금융취약도지수,국민기초생활보장_독거노인_수급자비율_x,저소득_독거노인_비율,일반_독거노인_비율,국민기초생활보장_독거노인_수급자비율_y,국민기초생활보장_65~79세_수급자비율,국민기초생활보장_80세이상_수급자비율
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,2033.146277,64613.103723,783.196809,31.997,0.177093,0.059912,0.762996,0.177093,0.173333,0.184416
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,2294.994681,114478.763298,783.319149,50.099,0.100575,0.106322,0.793103,0.100575,0.105991,0.091603
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,2078.319149,73207.087766,779.869681,35.444,0.113839,0.049107,0.837054,0.113839,0.111913,0.116959
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,2078.478723,76142.292553,779.638298,36.854,0.125604,0.130435,0.743961,0.125604,0.124060,0.128378
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,2095.970745,74372.853723,774.949468,35.709,0.090909,0.038961,0.870130,0.090909,0.072917,0.120690


## 15.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_독거노인_2019_일반_비율.csv')

In [ ]:
a.head()

,구,이름,일반_독거노인_비율,일반_65~79세_비율,일반_80세이상_비율
0,동남구,목천읍,0.762996,0.792000,0.706494
1,동남구,풍세면,0.793103,0.843318,0.709924
2,동남구,광덕면,0.837054,0.862816,0.795322
3,동남구,북면,0.743961,0.789474,0.662162
4,동남구,성남면,0.870130,0.906250,0.810345


In [ ]:
a_subset = a.iloc[:, 1:]

In [ ]:
a_subset.head()

,이름,일반_독거노인_비율,일반_65~79세_비율,일반_80세이상_비율
0,목천읍,0.762996,0.792000,0.706494
1,풍세면,0.793103,0.843318,0.709924
2,광덕면,0.837054,0.862816,0.795322
3,북면,0.743961,0.789474,0.662162
4,성남면,0.870130,0.906250,0.810345


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,금융취약도지수,국민기초생활보장_독거노인_수급자비율_x,저소득_독거노인_비율,일반_독거노인_비율_x,국민기초생활보장_독거노인_수급자비율_y,국민기초생활보장_65~79세_수급자비율,국민기초생활보장_80세이상_수급자비율,일반_독거노인_비율_y,일반_65~79세_비율,일반_80세이상_비율
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,31.997,0.177093,0.059912,0.762996,0.177093,0.173333,0.184416,0.762996,0.792000,0.706494
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,50.099,0.100575,0.106322,0.793103,0.100575,0.105991,0.091603,0.793103,0.843318,0.709924
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,35.444,0.113839,0.049107,0.837054,0.113839,0.111913,0.116959,0.837054,0.862816,0.795322
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,36.854,0.125604,0.130435,0.743961,0.125604,0.124060,0.128378,0.743961,0.789474,0.662162
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,35.709,0.090909,0.038961,0.870130,0.090909,0.072917,0.120690,0.870130,0.906250,0.810345


## 16.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_독거노인_2019_저소득_비율.csv')

In [ ]:
a.head()

,구,이름,저소득_독거노인_비율,저소득_65~79세_비율,저소득_80세이상_비율
0,동남구,목천읍,0.059912,0.034667,0.109091
1,동남구,풍세면,0.106322,0.050691,0.198473
2,동남구,광덕면,0.049107,0.025271,0.087719
3,동남구,북면,0.130435,0.086466,0.209459
4,동남구,성남면,0.038961,0.020833,0.068966


In [ ]:
a_subset = a.iloc[:, 1:]

In [ ]:
a_subset.head()

,이름,저소득_독거노인_비율,저소득_65~79세_비율,저소득_80세이상_비율
0,목천읍,0.059912,0.034667,0.109091
1,풍세면,0.106322,0.050691,0.198473
2,광덕면,0.049107,0.025271,0.087719
3,북면,0.130435,0.086466,0.209459
4,성남면,0.038961,0.020833,0.068966


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,일반_독거노인_비율_x,국민기초생활보장_독거노인_수급자비율_y,국민기초생활보장_65~79세_수급자비율,국민기초생활보장_80세이상_수급자비율,일반_독거노인_비율_y,일반_65~79세_비율,일반_80세이상_비율,저소득_독거노인_비율_y,저소득_65~79세_비율,저소득_80세이상_비율
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.762996,0.177093,0.173333,0.184416,0.762996,0.792000,0.706494,0.059912,0.034667,0.109091
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.793103,0.100575,0.105991,0.091603,0.793103,0.843318,0.709924,0.106322,0.050691,0.198473
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.837054,0.113839,0.111913,0.116959,0.837054,0.862816,0.795322,0.049107,0.025271,0.087719
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.743961,0.125604,0.124060,0.128378,0.743961,0.789474,0.662162,0.130435,0.086466,0.209459
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.870130,0.090909,0.072917,0.120690,0.870130,0.906250,0.810345,0.038961,0.020833,0.068966


## 16.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_독거노인_가구.csv')

In [ ]:
a.head()

,구,구분,이름,관할 법정리,전체노인인구_합계,전체노인_남자,전체노인_여자,65세남자,66세남자,67세남자,...,105세여자,106세여자,107세여자,108세여자,109세여자,110세이상 여자,독거노인_남자,독거노인_여자,독거노인_합계,독거노인_비율
0,동남구,읍·면,목천읍,"교천리,교촌리,남화리,덕전리,도장리,동리,동평리,삼성리,서리,서흥리,석천리,소사리,...",5067,2787,2280,67,59,53,...,0,0,0,0,0,0,643,1067,1710,0.337478
1,동남구,읍·면,풍세면,"가송리,남관리,두남리,미죽리,보성리,삼태리,용정리,풍서리",2422,1447,975,30,14,15,...,0,0,0,0,0,0,223,343,566,0.233691
2,동남구,읍·면,광덕면,"광덕리,대덕리,대평리,매당리,무학리,산원리,신덕리,신흥리,원덕리,지장리,행정리",1061,530,531,22,14,16,...,0,0,0,0,0,0,201,336,537,0.506126
3,동남구,읍·면,북면,"납안리,대평리,매송리,명덕리,사담리,상동리,양곡리,연춘리,오곡리,용암리,운용리,은지...",1342,745,597,38,26,9,...,0,0,0,0,0,0,252,355,607,0.452310
4,동남구,읍·면,성남면,"대정리,대화리,대흥리,봉양리,석곡리,신덕리,신사리,용원리,화성리",782,432,350,13,14,20,...,0,0,0,0,0,0,170,231,401,0.512788


In [ ]:
a_subset = a.iloc[:, [2, -2, -1]]

In [ ]:
a_subset.head()

,이름,독거노인_합계,독거노인_비율
0,목천읍,1710,0.337478
1,풍세면,566,0.233691
2,광덕면,537,0.506126
3,북면,607,0.452310
4,성남면,401,0.512788


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,국민기초생활보장_65~79세_수급자비율,국민기초생활보장_80세이상_수급자비율,일반_독거노인_비율_y,일반_65~79세_비율,일반_80세이상_비율,저소득_독거노인_비율_y,저소득_65~79세_비율,저소득_80세이상_비율,독거노인_합계,독거노인_비율
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.173333,0.184416,0.762996,0.792000,0.706494,0.059912,0.034667,0.109091,1710,0.337478
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.105991,0.091603,0.793103,0.843318,0.709924,0.106322,0.050691,0.198473,566,0.233691
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.111913,0.116959,0.837054,0.862816,0.795322,0.049107,0.025271,0.087719,537,0.506126
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.124060,0.128378,0.743961,0.789474,0.662162,0.130435,0.086466,0.209459,607,0.452310
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.072917,0.120690,0.870130,0.906250,0.810345,0.038961,0.020833,0.068966,401,0.512788


## 17.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_독거노인_비율.csv')

In [ ]:
a.head()

,구,구분,이름,계,남자,여자,고령_남자,고령_여자,고령_1인가구수,고령_비율
0,동남구,읍·면,목천읍,5067,2787,2280,643,1067,1710,0.337478
1,동남구,읍·면,풍세면,2422,1447,975,223,343,566,0.233691
2,동남구,읍·면,광덕면,1061,530,531,201,336,537,0.506126
3,동남구,읍·면,북면,1342,745,597,252,355,607,0.452310
4,동남구,읍·면,성남면,782,432,350,170,231,401,0.512788


In [ ]:
a_subset = a.iloc[:, [2,-2,-1]]

In [ ]:
a_subset.head()

,이름,고령_1인가구수,고령_비율
0,목천읍,1710,0.337478
1,풍세면,566,0.233691
2,광덕면,537,0.506126
3,북면,607,0.452310
4,성남면,401,0.512788


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,일반_독거노인_비율_y,일반_65~79세_비율,일반_80세이상_비율,저소득_독거노인_비율_y,저소득_65~79세_비율,저소득_80세이상_비율,독거노인_합계,독거노인_비율,고령_1인가구수,고령_비율
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.762996,0.792000,0.706494,0.059912,0.034667,0.109091,1710,0.337478,1710,0.337478
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.793103,0.843318,0.709924,0.106322,0.050691,0.198473,566,0.233691,566,0.233691
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.837054,0.862816,0.795322,0.049107,0.025271,0.087719,537,0.506126,537,0.506126
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.743961,0.789474,0.662162,0.130435,0.086466,0.209459,607,0.452310,607,0.452310
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.870130,0.906250,0.810345,0.038961,0.020833,0.068966,401,0.512788,401,0.512788


## 18.

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/ESAA OB 2조/방학 프로젝트/데이터/최종 세부지표/세부지표_천안시_독거노인_증감인원.csv')

In [ ]:
a.head()

,구,이름,계_증감,계_2025,계_2019,65~79세_2019,80세 이상_2019
0,동남구,목천읍,575,1710,1135,750,385
1,동남구,풍세면,218,566,348,217,131
2,동남구,광덕면,89,537,448,277,171
3,동남구,북면,193,607,414,266,148
4,동남구,성남면,93,401,308,192,116


In [ ]:
a_subset = a.iloc[:, 1:]

In [ ]:
a_subset.head()

,이름,계_증감,계_2025,계_2019,65~79세_2019,80세 이상_2019
0,목천읍,575,1710,1135,750,385
1,풍세면,218,566,348,217,131
2,광덕면,89,537,448,277,171
3,북면,193,607,414,266,148
4,성남면,93,401,308,192,116


In [ ]:
data = data.merge(a_subset, on='이름', how='left')

In [ ]:
data.head()

,구,구분,이름,전체,65세 이상,고령화율(단위: 퍼센트),노인1천명당_재가복지시설수,노인1천명당_주거복지시설수,노인1천명당_의료복지시설수,노인1천명당_여가복지시설수_경로당,...,저소득_80세이상_비율,독거노인_합계,독거노인_비율,고령_1인가구수,고령_비율,계_증감,계_2025,계_2019,65~79세_2019,80세 이상_2019
0,동남구,읍·면,목천읍,22292,4924,22.1,2.233956,0.406174,1.421608,8.529651,...,0.109091,1710,0.337478,1710,0.337478,575,1710,1135,750,385
1,동남구,읍·면,풍세면,10401,2044,19.7,2.935421,0.000000,2.446184,12.230920,...,0.198473,566,0.233691,566,0.233691,218,566,348,217,131
2,동남구,읍·면,광덕면,4004,1722,43.0,1.742160,0.000000,1.161440,21.486643,...,0.087719,537,0.506126,537,0.506126,89,537,448,277,171
3,동남구,읍·면,북면,4109,1641,39.9,1.828154,0.000000,0.609385,15.234613,...,0.209459,607,0.452310,607,0.452310,193,607,414,266,148
4,동남구,읍·면,성남면,2812,1281,45.6,2.341920,0.000000,0.000000,23.419204,...,0.068966,401,0.512788,401,0.512788,93,401,308,192,116


In [ ]:
data.columns

Index(['구', '구분', '이름', '전체', '65세 이상', '고령화율(단위: 퍼센트)', '노인1천명당_재가복지시설수',
       '노인1천명당_주거복지시설수', '노인1천명당_의료복지시설수', '노인1천명당_여가복지시설수_경로당',
       '노인1천명당_여가복지시설수_노인교실', '노인1천명당_노인돌봄서비스수행기관수', '노인1천명당_무료급식소수',
       '노인1천명당_치매안심센터수', '노인1천명당_자립형일자리수행기관수', '고령자비율', '고령자밀집도지수',
       '복지시설다양성지수', '교통취약노인수', '복지서비스잠재수요지수', '사고건수', '보호구역 수', '기준연월',
       '월경제활동인구수_65_69_x', '월경제활동인구수_70세이상_x', '월대출평균잔액_65_69',
       '월대출평균잔액_70세이상', '월소득평균_65_69', '월소득평균_70세이상', '월신용평점평균_65_69',
       '월신용평점평균_70세이상', '월카드총이용금액평균_65_69', '월카드총이용금액평균_70세이상',
       '경제활동참가율_65_69', '경제활동참가율_70세이상', '고령인구수_65_69', '고령인구수_70세이상',
       '월경제활동인구수_65_69_y', '월경제활동인구수_70세이상_y', '경제활동참가율_전체', '월소득평균',
       '월대출평균잔액', '월신용평점평균', '금융취약도지수', '국민기초생활보장_독거노인_수급자비율_x',
       '저소득_독거노인_비율_x', '일반_독거노인_비율_x', '국민기초생활보장_독거노인_수급자비율_y',
       '국민기초생활보장_65~79세_수급자비율', '국민기초생활보장_80세이상_수급자비율', '일반_독거노인_비율_y',
       '일반_65~79세_비율', '일반_80세이상_비율', '저소득_독거노인_비율_y', '저소득_65~79세_비율',
       '저소득_80세이상_비율', '독거노인_합계', '독

In [ ]:
[x for x in data.columns if '_x' in x]

['월경제활동인구수_65_69_x',
 '월경제활동인구수_70세이상_x',
 '국민기초생활보장_독거노인_수급자비율_x',
 '저소득_독거노인_비율_x',
 '일반_독거노인_비율_x']

In [ ]:
(data['월경제활동인구수_65_69_x'] == data['월경제활동인구수_65_69_y']).all()

np.True_

In [ ]:
(data['월경제활동인구수_70세이상_x'] == data['월경제활동인구수_70세이상_y']).all()

np.True_

In [ ]:
(data['국민기초생활보장_독거노인_수급자비율_x'] == data['국민기초생활보장_독거노인_수급자비율_y']).all()

np.True_

In [ ]:
(data['저소득_독거노인_비율_x'] == data['저소득_독거노인_비율_y']).all()

np.True_

In [ ]:
(data['일반_독거노인_비율_x'] == data['일반_독거노인_비율_y']).all()

np.True_

In [ ]:
# 1. _y 칼럼 제거
data = data.drop(columns=[col for col in data.columns if col.endswith('_y')])

# 2. _x 칼럼명을 원래 이름으로 리네이밍
data = data.rename(columns={col: col.replace('_x', '') for col in data.columns if col.endswith('_x')})

In [ ]:
data.columns

Index(['구', '구분', '이름', '전체', '65세 이상', '고령화율(단위: 퍼센트)', '노인1천명당_재가복지시설수',
       '노인1천명당_주거복지시설수', '노인1천명당_의료복지시설수', '노인1천명당_여가복지시설수_경로당',
       '노인1천명당_여가복지시설수_노인교실', '노인1천명당_노인돌봄서비스수행기관수', '노인1천명당_무료급식소수',
       '노인1천명당_치매안심센터수', '노인1천명당_자립형일자리수행기관수', '고령자비율', '고령자밀집도지수',
       '복지시설다양성지수', '교통취약노인수', '복지서비스잠재수요지수', '사고건수', '보호구역 수', '기준연월',
       '월경제활동인구수_65_69', '월경제활동인구수_70세이상', '월대출평균잔액_65_69', '월대출평균잔액_70세이상',
       '월소득평균_65_69', '월소득평균_70세이상', '월신용평점평균_65_69', '월신용평점평균_70세이상',
       '월카드총이용금액평균_65_69', '월카드총이용금액평균_70세이상', '경제활동참가율_65_69',
       '경제활동참가율_70세이상', '고령인구수_65_69', '고령인구수_70세이상', '경제활동참가율_전체', '월소득평균',
       '월대출평균잔액', '월신용평점평균', '금융취약도지수', '국민기초생활보장_독거노인_수급자비율', '저소득_독거노인_비율',
       '일반_독거노인_비율', '국민기초생활보장_65~79세_수급자비율', '국민기초생활보장_80세이상_수급자비율',
       '일반_65~79세_비율', '일반_80세이상_비율', '저소득_65~79세_비율', '저소득_80세이상_비율',
       '독거노인_합계', '독거노인_비율', '고령_1인가구수', '고령_비율', '계_증감', '계_2025', '계_2019',
       '65~79세_2019', '80세 이상_2019'],
      dtype='object')

In [ ]:
from google.colab import files

# 1. CSV 파일로 저장
data.to_csv('최종_병합_데이터.csv', index=False, encoding='utf-8-sig')

# 2. 저장한 파일 다운로드
files.download('최종_병합_데이터.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>